In [5]:
from matplotlib.font_manager import weight_dict
from click import style
import torch
import torchvision.models as models
import torch.nn.functional as F
import torch.nn as nn
import numpy as np
import numbers
import math
import cv2
from PIL import Image
from torchvision.transforms import Compose, ToTensor, Normalize, Resize, ToPILImage
import time

from torchvision import utils as vutils


t0 = time.time()


# 导入预设模型
model = models.vgg19(weights=models.VGG19_Weights.IMAGENET1K_V1).cuda()
batch_size = 1

# 将VGG当作特征提取器, 无须训练其中参数
for params in model.parameters():
    params.requires_grad = False

# 将模型设置为评估模式
model.eval()

# # 这段代码是用来对图像进行标准化的
# 将图像从 [0, 1] 范围拉伸到 [-0.5, 0.5] 范围，方便网络训练
# 具体做法是减去均值，再除以标准差
# 所以图像的数值范围变成了 [-0.5, 0.5]
# mu 是均值，也就是 RGB 三个通道的均值
# 这里的均值是根据ImageNet数据集计算出来的
# 具体来说，是对ImageNet数据集中所有图像的三个通道的平均值计算得到的
# 这个操作的意义在于，对图像进行标准化后，网络训练的初始权重就能够适应图像的这种分布
mu = torch.Tensor([0.485, 0.456, 0.406]).unsqueeze(dim=-1).unsqueeze(-1).cuda()
print(mu.size())

std = torch.Tensor([0.229, 0.224, 0.225]).unsqueeze(-1).unsqueeze(-1).cuda()
unnormalize = lambda x: x*std + mu
normalize = lambda x: (x-mu)/std
transform_test = Compose([
    Resize((512,512)),
    ToTensor(),
])

content_img_path = '/home/zxt/Python_area/for_wu_ding_minecraft_modules/datas/2_preprocessed_data/preprocessed_tomatoes_edge.png'
style_img_path = '/home/zxt/Python_area/for_wu_ding_minecraft_modules/datas/1_origin_data/bamboo_leaf.jpg'



content_img = Image.open(content_img_path)
content_img = content_img.convert('RGB')
image_size = content_img.size
content_img = transform_test(content_img)
# 将content_img转换为Image类的对象. 并保存



print(content_img.size())
content_img = content_img.squeeze(0).cuda()
print(content_img.size())

style_img = Image.open(fp=style_img_path)
style_img = transform_test(style_img)
vutils.save_image(style_img, './test.png')

print(style_img.size())
style_img = style_img.squeeze(0).cuda()
print(style_img.size())


torch.Size([3, 1, 1])
torch.Size([3, 512, 512])
torch.Size([3, 512, 512])
torch.Size([4, 512, 512])
torch.Size([4, 512, 512])
